DATASET liiklusonnetused

In [2]:
import pandas as pd

In [4]:
df_original = pd.read_csv('liiklusonnetused_until_2021.csv')
print(df_original.columns)

Index(['Juhtumi nr', 'Toimumisaeg', 'Isikuid', 'Hukkunuid', 'Vigastatuid',
       'Sõidukeid', 'Aadress (PPA)', 'Maja nr (PPA)', 'Tänav (PPA)',
       'Ristuv tänav (PPA)', 'Maakond (PPA)', 'Omavalitsus (PPA)',
       'Asustus (PPA)', 'Asula', 'Liiklusõnnetuse liik [1]',
       'Liiklusõnnetuse liik [3]', 'Kergliikurijuhi osalusel',
       'Jalakäija osalusel', 'Kaassõitja osalusel',
       'Maastikusõiduki juhi osalusel',
       'Eaka (65+) mootorsõidukijuhi osalusel', 'Bussijuhi osalusel',
       'Veoautojuhi osalusel', 'Ühissõidukijuhi osalusel',
       'Sõiduautojuhi osalusel', 'Mootorratturi osalusel',
       'Mopeedijuhi osalusel', 'Jalgratturi osalusel', 'Alaealise osalusel',
       'Turvavarustust mitte kasutanud isiku osalusel',
       'Esmase juhiloa omaniku osalusel', 'Mootorsõidukijuhi osalusel',
       'Tüüpskeemi nr', 'Tüüpskeem [2]', 'Tee tüüp [1]', 'Tee tüüp [2]',
       'Tee element [1]', 'Tee element [2]', 'Tee objekt [2]', 'Kurvilisus',
       'Tee tasasus', 'Tee sei

In [5]:
#Keeping only neccesary columns
columns_to_keep = ['Juhtumi nr', 'Toimumisaeg', 'Asula', 'Liiklusõnnetuse liik [3]','GPS X', 'GPS Y']
df = df_original[columns_to_keep]
print(df.head())

      Juhtumi nr       Toimumisaeg Asula          Liiklusõnnetuse liik [3]  \
0  2000140000057  24.10.2014 08:45   JAH            Kokkupõrge jalakäijaga   
1  2000140000067  24.10.2014 13:45   JAH  Kokkupõrge ees liikuva sõidukiga   
2  2000140000123           8/11/14   JAH  Kokkupõrge ees liikuva sõidukiga   
3  2000140000235  17.11.2014 17:32    EI  Kokkupõrge vastutuleva sõidukiga   
4  2000150000442  28.04.2015 07:55   JAH      Kokkupõrge sõidukiga küljelt   

       GPS X     GPS Y  
0  6588678.0  542647.0  
1  6589522.0  541467.0  
2  6593961.0  547646.0  
3  6569324.0  516628.0  
4  6586430.0  541953.0  


In [15]:
#Delete all rows that does not have time
num_rows = df.shape[0]
df_clean = df[~df['Toimumisaeg'].str.contains(r'\d+/\d+/\d{2}')]
num_rows_new = df_clean.shape[0]
#print(df_clean.head())
#print(num_rows) 
#print(num_rows_new) #15702 rows originally and now 9322, its almost half the data :(

# Convert "Toimumisaeg" column to datetime format
df_clean['Toimumisaeg'] = pd.to_datetime(df_clean['Toimumisaeg'], dayfirst=True)
#print(df_clean.head())

# Extract day, month, year, hour, and minute into separate columns
df_clean['Day'] = df_clean['Toimumisaeg'].dt.day
df_clean['Month'] = df_clean['Toimumisaeg'].dt.month
df_clean['Year'] = df_clean['Toimumisaeg'].dt.year
df_clean['Hour'] = df_clean['Toimumisaeg'].dt.hour
df_clean['Minute'] = df_clean['Toimumisaeg'].dt.minute

print(df_clean.head())

      Juhtumi nr         Toimumisaeg Asula          Liiklusõnnetuse liik [3]  \
0  2000140000057 2014-10-24 08:45:00   JAH            Kokkupõrge jalakäijaga   
1  2000140000067 2014-10-24 13:45:00   JAH  Kokkupõrge ees liikuva sõidukiga   
3  2000140000235 2014-11-17 17:32:00    EI  Kokkupõrge vastutuleva sõidukiga   
4  2000150000442 2015-04-28 07:55:00   JAH      Kokkupõrge sõidukiga küljelt   
6  2210140001932 2014-10-24 14:00:00    EI  Kokkupõrge vastutuleva sõidukiga   

       GPS X     GPS Y  Day  Month  Year  Hour  Minute  
0  6588678.0  542647.0   24     10  2014     8      45  
1  6589522.0  541467.0   24     10  2014    13      45  
3  6569324.0  516628.0   17     11  2014    17      32  
4  6586430.0  541953.0   28      4  2015     7      55  
6  6575538.0  529034.0   24     10  2014    14       0  


C:\Users\jarve\AppData\Local\Temp\ipykernel_12848\3039620202.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Toimumisaeg'] = pd.to_datetime(df_clean['Toimumisaeg'], dayfirst=True)
C:\Users\jarve\AppData\Local\Temp\ipykernel_12848\3039620202.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Day'] = df_clean['Toimumisaeg'].dt.day
C:\Users\jarve\AppData\Local\Temp\ipykernel_12848\3039620202.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [16]:
#CHANGING COLUMN NAMES

# Create a dictionary to map the old column names to the new column names
column_mapping = {
    'Juhtumi nr': 'Juhtumi_nr',
    'Liiklusõnnetuse liik [3]': 'Liiklusonnetuse_liik',
}

# Use the 'rename()' method to change the column names
df_clean = df_clean.rename(columns=column_mapping)

# Print the modified DataFrame with the new column names
print(df_clean.head())

      Juhtumi_nr         Toimumisaeg Asula              Liiklusonnetuse_liik  \
0  2000140000057 2014-10-24 08:45:00   JAH            Kokkupõrge jalakäijaga   
1  2000140000067 2014-10-24 13:45:00   JAH  Kokkupõrge ees liikuva sõidukiga   
3  2000140000235 2014-11-17 17:32:00    EI  Kokkupõrge vastutuleva sõidukiga   
4  2000150000442 2015-04-28 07:55:00   JAH      Kokkupõrge sõidukiga küljelt   
6  2210140001932 2014-10-24 14:00:00    EI  Kokkupõrge vastutuleva sõidukiga   

       GPS X     GPS Y  Day  Month  Year  Hour  Minute  
0  6588678.0  542647.0   24     10  2014     8      45  
1  6589522.0  541467.0   24     10  2014    13      45  
3  6569324.0  516628.0   17     11  2014    17      32  
4  6586430.0  541953.0   28      4  2015     7      55  
6  6575538.0  529034.0   24     10  2014    14       0  


In [18]:
#MISSING VALUES
# Check for missing values in the DataFrame
missing_values = df_clean.isnull()

missing_counts = df.isnull().sum()
print(missing_counts) #2244-l real ei ole GPS koordinaate

Juhtumi nr                     0
Toimumisaeg                    0
Asula                          0
Liiklusõnnetuse liik [3]       0
GPS X                       2244
GPS Y                       2249
dtype: int64
